# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction JSON Dataset to CSV Dataset Conversion

## UPDATE: There was a bug in previous version, in train set all measurement values were flattened to the first 2400\*68 entries followed by 2400\*39 nan. This is updated to flatten correctly with 68 entries and 39 nan for each id.

I think that for EDA, a flat file based representation would be good. Hence I am converting the json hierarchical dataset of the competition to CSV format popular among Kagglers.

## References: My public notebooks for OpenVaccine

* [Flatten JSON Data](https://www.kaggle.com/arunprathap/openvaccine-flatten-json-data)

* [Feature Correlation, Importance](https://www.kaggle.com/arunprathap/openvaccine-feature-correlation-importance)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from pandas.io.json import json_normalize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Read train and test json

In [4]:
train_full = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
train_full.shape

(2400, 19)

In [5]:
test_full = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
test_full.shape

(3634, 7)

In [6]:
train_full.head().T

,0,1,2,3,4
index,0,1,2,3,4
id,id_001f94081,id_0049f53ba,id_006f36f57,id_0082d463b,id_0087940f4
sequence,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...
structure,.....((((((.......)))).)).((.....((..((((((......,.....(((((((((((((((((((((((....)))))))))).)))...,.....((((.((.....((((.(((.....)))..((((......)...,......((((((((((((((((......))))))))))))))))((...,.....(((((((.((((((((((((.(((((((((....)))))))...
predicted_loop_type,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...
signal_to_noise,6.894,0.193,8.8,0.104,0.423
SN_filter,1,0,1,0,0
seq_length,107,107,107,107,107
seq_scored,68,68,68,68,68
reactivity_error,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[0.0931, 0.13290000000000002, 0.11280000000000...","[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000..."


In [7]:
test_full.head().T

,0,1,2,3,4
index,0,1,2,3,4
id,id_00073f8be,id_000ae4237,id_00131c573,id_00181fd34,id_0020473f7
sequence,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...
structure,......((((((((((.(((((.....))))))))((((((((......,.....((((..((((((...(((((.....((((....)))).......,...........((.(((.(.(..((..((..((((...))))..))...,......((((((((((....))))))))))((((((..((.(((.....,.....(((((((((((((((((((((((((((((((((((((((((...
predicted_loop_type,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...
seq_length,107,130,107,107,130
seq_scored,68,91,68,68,91


Not so intuitive, right. Lets see if we can flatten it.

In [8]:
train_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/train.json', 'r')]
test_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/test.json', 'r')]

In [9]:
def preprocess_json(input_json, is_train):
    
    for index,json_ in enumerate(input_json):
        json_['step'] = list(range(json_['seq_length']))
        json_['sequence'] = list(json_['sequence'])
        json_['structure'] = list(json_['structure'])
        json_['predicted_loop_type'] = list(json_['predicted_loop_type'])
        if os.path.exists('../input/stanford-covid-vaccine/bpps/'+json_['id']+'.npy'):
            np_array = np.load('../input/stanford-covid-vaccine/bpps/'+json_['id']+'.npy')
            json_['unpaired_probability'] = list(1-sum(np_array))
        else:
            print('bpps not found')
        if is_train:
            append_list = [np.nan for i in range(39)]
            json_['reactivity'].extend(append_list)
            json_['reactivity_error'].extend(append_list)
            json_['deg_Mg_pH10'].extend(append_list)
            json_['deg_error_Mg_pH10'].extend(append_list)
            json_['deg_pH10'].extend(append_list)
            json_['deg_error_pH10'].extend(append_list)
            json_['deg_Mg_50C'].extend(append_list)
            json_['deg_error_Mg_50C'].extend(append_list)
            json_['deg_50C'].extend(append_list)
            json_['deg_error_50C'].extend(append_list)
                
preprocess_json(train_json, True)
preprocess_json(test_json, False)

In [10]:
def process_json(input_json, is_train):
    if is_train:
        train = pd.json_normalize(data = input_json, 
                                record_path ='step',  
                                meta =['id','seq_length','seq_scored','SN_filter','signal_to_noise']) 
    else:
        train = pd.json_normalize(data = input_json, 
                                record_path ='step',  
                                meta =['id','seq_length','seq_scored']) 
    train.rename(columns={0:'step'}, inplace=True)
    train['unpaired_probability'] = pd.json_normalize(data = input_json, 
                                record_path ='unpaired_probability'
                                            )
    train['sequence'] = pd.json_normalize(data = input_json, 
                                record_path ='sequence'
                                            )
    train['structure'] = pd.json_normalize(data = input_json, 
                                record_path ='structure'
                                            )
    train['predicted_loop_type'] = pd.json_normalize(data = input_json, 
                                record_path ='predicted_loop_type'
                                            )
    if is_train:
        train['reactivity'] = pd.json_normalize(data = input_json, 
                                        record_path ='reactivity'
                                                    )
        train['reactivity_error'] = pd.json_normalize(data = input_json, 
                                        record_path ='reactivity_error'
                                                    )
        train['deg_Mg_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_Mg_pH10'
                                                    )
        train['deg_error_Mg_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_Mg_pH10'
                                                    )
        train['deg_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_pH10',
                                                    )
        train['deg_error_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_pH10',
                                                    )
        train['deg_Mg_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_Mg_50C',
                                                    )
        train['deg_error_Mg_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_Mg_50C',
                                                    )
        train['deg_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_50C',
                                                    )
        train['deg_error_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_50C',
                                                    )
    
    train.set_index(['id','step'], inplace=True)
    return train

X_train = process_json(train_json, True)
X_test = process_json(test_json, False)
print(X_train.shape, X_test.shape)

(256800, 18) (457953, 6)


So we got 256800 = 2400\*107 rows. But since the measured values are there only for first 68 pairs, we will have NaN on the remaining columns for these entries. This is not cleaned here since the user can easily remove those with dropna if he wants only 68 entries or can fill with dummy values if he wants to use all 107. So we will have 2400\*39 = 96300 NaN values in these columns.

In [11]:
X_train.isnull().any()

seq_length              False
seq_scored              False
SN_filter               False
signal_to_noise         False
unpaired_probability    False
sequence                False
structure               False
predicted_loop_type     False
reactivity               True
reactivity_error         True
deg_Mg_pH10              True
deg_error_Mg_pH10        True
deg_pH10                 True
deg_error_pH10           True
deg_Mg_50C               True
deg_error_Mg_50C         True
deg_50C                  True
deg_error_50C            True
dtype: bool

In [12]:
X_train.isnull().sum()

seq_length                  0
seq_scored                  0
SN_filter                   0
signal_to_noise             0
unpaired_probability        0
sequence                    0
structure                   0
predicted_loop_type         0
reactivity              93600
reactivity_error        93600
deg_Mg_pH10             93600
deg_error_Mg_pH10       93600
deg_pH10                93600
deg_error_pH10          93600
deg_Mg_50C              93600
deg_error_Mg_50C        93600
deg_50C                 93600
deg_error_50C           93600
dtype: int64

In [13]:
X_train.head()

seq_length seq_scored SN_filter signal_to_noise  \
id           step                                                   
id_001f94081 0           107         68         1           6.894   
             1           107         68         1           6.894   
             2           107         68         1           6.894   
             3           107         68         1           6.894   
             4           107         68         1           6.894   

                   unpaired_probability sequence structure  \
id           step                                            
id_001f94081 0                 0.801458        G         .   
             1                 0.816288        G         .   
             2                 0.939998        A         .   
             3                 0.986878        A         .   
             4                 0.988720        A         .   

                  predicted_loop_type  reactivity  reactivity_error  \
id           step                                                     
id_001f94081 0                      E      0.3297            0.1359   
             1                      E      1.5693            0.2070   
             2                      E      1.1227            0.1633   
             3                      E      0.8686            0.1452   
             4                      E      0.7217            0.1314   

                   deg_Mg_pH10  deg_error_Mg_pH10  deg_pH10  deg_error_pH10  \
id           step                                                             
id_001f94081 0          0.7556             0.2613    2.3375          0.2631   
             1          2.9830             0.3842    3.5060          0.2860   
             2          0.2526             0.1372    0.3008          0.0964   
             3          1.3789             0.2531    1.0108          0.1574   
             4          0.6376             0.1798    0.2635          0.1000   

                   deg_Mg_50C  deg_error_Mg_50C  deg_50C  deg_error_50C  
id           step                                                        
id_001f94081 0         0.3581            0.1501   0.6382         0.2167  
             1         2.9683            0.2750   3.4773         0.3475  
             2         0.2589            0.0947   0.9988         0.1880  
             3         1.4552            0.1866   1.3228         0.2124  
             4         0.7244            0.1369   0.7877         0.1703

In [14]:
print(X_test[X_test.seq_length==107].shape, X_test[X_test.seq_length==130].shape)

(67303, 6) (390650, 6)


Test set has 629\*107 = 67303 + 3005\*130 = 390650 entries corresponding to 3634 rows in test set.

In [15]:
print(X_test.isnull().any())

seq_length              False
seq_scored              False
unpaired_probability    False
sequence                False
structure               False
predicted_loop_type     False
dtype: bool


There are no NaN entries as expected.

In [16]:
X_test.head()

seq_length seq_scored  unpaired_probability sequence  \
id           step                                                        
id_00073f8be 0           107         68              0.962984        G   
             1           107         68              0.987545        G   
             2           107         68              0.997380        A   
             3           107         68              0.999286        A   
             4           107         68              0.995819        A   

                  structure predicted_loop_type  
id           step                                
id_00073f8be 0            .                   E  
             1            .                   E  
             2            .                   E  
             3            .                   E  
             4            .                   E

### Write to CSV file

In [17]:
X_train.to_csv('train.csv')
X_test.to_csv('test.csv')

In [18]:
!cp ../input/stanford-covid-vaccine/sample_submission.csv sample_submission.csv 

### Read back and verify shape and contents

In [19]:
test_train = pd.read_csv('train.csv', index_col=[0,1])
print(X_train.dtypes)
print(test_train.dtypes)

seq_length               object
seq_scored               object
SN_filter                object
signal_to_noise          object
unpaired_probability    float64
sequence                 object
structure                object
predicted_loop_type      object
reactivity              float64
reactivity_error        float64
deg_Mg_pH10             float64
deg_error_Mg_pH10       float64
deg_pH10                float64
deg_error_pH10          float64
deg_Mg_50C              float64
deg_error_Mg_50C        float64
deg_50C                 float64
deg_error_50C           float64
dtype: object
seq_length                int64
seq_scored                int64
SN_filter               float64
signal_to_noise         float64
unpaired_probability    float64
sequence                 object
structure                object
predicted_loop_type      object
reactivity              float64
reactivity_error        float64
deg_Mg_pH10             float64
deg_error_Mg_pH10       float64
deg_pH10                fl

Lets compare the columns of same type (float64 not compared as floating point numbers will not match due to precision)

In [20]:
cols = ['sequence','structure','predicted_loop_type']
print(X_train.shape, test_train.shape, test_train[cols].equals(X_train[cols]))

(256800, 18) (256800, 18) True


In [21]:
X_train.head()

seq_length seq_scored SN_filter signal_to_noise  \
id           step                                                   
id_001f94081 0           107         68         1           6.894   
             1           107         68         1           6.894   
             2           107         68         1           6.894   
             3           107         68         1           6.894   
             4           107         68         1           6.894   

                   unpaired_probability sequence structure  \
id           step                                            
id_001f94081 0                 0.801458        G         .   
             1                 0.816288        G         .   
             2                 0.939998        A         .   
             3                 0.986878        A         .   
             4                 0.988720        A         .   

                  predicted_loop_type  reactivity  reactivity_error  \
id           step                                                     
id_001f94081 0                      E      0.3297            0.1359   
             1                      E      1.5693            0.2070   
             2                      E      1.1227            0.1633   
             3                      E      0.8686            0.1452   
             4                      E      0.7217            0.1314   

                   deg_Mg_pH10  deg_error_Mg_pH10  deg_pH10  deg_error_pH10  \
id           step                                                             
id_001f94081 0          0.7556             0.2613    2.3375          0.2631   
             1          2.9830             0.3842    3.5060          0.2860   
             2          0.2526             0.1372    0.3008          0.0964   
             3          1.3789             0.2531    1.0108          0.1574   
             4          0.6376             0.1798    0.2635          0.1000   

                   deg_Mg_50C  deg_error_Mg_50C  deg_50C  deg_error_50C  
id           step                                                        
id_001f94081 0         0.3581            0.1501   0.6382         0.2167  
             1         2.9683            0.2750   3.4773         0.3475  
             2         0.2589            0.0947   0.9988         0.1880  
             3         1.4552            0.1866   1.3228         0.2124  
             4         0.7244            0.1369   0.7877         0.1703

In [22]:
test_train.head()

seq_length  seq_scored  SN_filter  signal_to_noise  \
id           step                                                       
id_001f94081 0            107          68        1.0            6.894   
             1            107          68        1.0            6.894   
             2            107          68        1.0            6.894   
             3            107          68        1.0            6.894   
             4            107          68        1.0            6.894   

                   unpaired_probability sequence structure  \
id           step                                            
id_001f94081 0                 0.801458        G         .   
             1                 0.816288        G         .   
             2                 0.939998        A         .   
             3                 0.986878        A         .   
             4                 0.988720        A         .   

                  predicted_loop_type  reactivity  reactivity_error  \
id           step                                                     
id_001f94081 0                      E      0.3297            0.1359   
             1                      E      1.5693            0.2070   
             2                      E      1.1227            0.1633   
             3                      E      0.8686            0.1452   
             4                      E      0.7217            0.1314   

                   deg_Mg_pH10  deg_error_Mg_pH10  deg_pH10  deg_error_pH10  \
id           step                                                             
id_001f94081 0          0.7556             0.2613    2.3375          0.2631   
             1          2.9830             0.3842    3.5060          0.2860   
             2          0.2526             0.1372    0.3008          0.0964   
             3          1.3789             0.2531    1.0108          0.1574   
             4          0.6376             0.1798    0.2635          0.1000   

                   deg_Mg_50C  deg_error_Mg_50C  deg_50C  deg_error_50C  
id           step                                                        
id_001f94081 0         0.3581            0.1501   0.6382         0.2167  
             1         2.9683            0.2750   3.4773         0.3475  
             2         0.2589            0.0947   0.9988         0.1880  
             3         1.4552            0.1866   1.3228         0.2124  
             4         0.7244            0.1369   0.7877         0.1703

In [23]:
test_test = pd.read_csv('test.csv', index_col=[0,1])
print(X_test.dtypes)
print(test_test.dtypes)

seq_length               object
seq_scored               object
unpaired_probability    float64
sequence                 object
structure                object
predicted_loop_type      object
dtype: object
seq_length                int64
seq_scored                int64
unpaired_probability    float64
sequence                 object
structure                object
predicted_loop_type      object
dtype: object


In [24]:
cols = ['sequence','structure','predicted_loop_type']
print(X_test.shape, test_test.shape, test_test[cols].equals(X_test[cols]))

(457953, 6) (457953, 6) True


In [25]:
X_test.head()

seq_length seq_scored  unpaired_probability sequence  \
id           step                                                        
id_00073f8be 0           107         68              0.962984        G   
             1           107         68              0.987545        G   
             2           107         68              0.997380        A   
             3           107         68              0.999286        A   
             4           107         68              0.995819        A   

                  structure predicted_loop_type  
id           step                                
id_00073f8be 0            .                   E  
             1            .                   E  
             2            .                   E  
             3            .                   E  
             4            .                   E

In [26]:
test_test.head()

seq_length  seq_scored  unpaired_probability sequence  \
id           step                                                          
id_00073f8be 0            107          68              0.962984        G   
             1            107          68              0.987545        G   
             2            107          68              0.997380        A   
             3            107          68              0.999286        A   
             4            107          68              0.995819        A   

                  structure predicted_loop_type  
id           step                                
id_00073f8be 0            .                   E  
             1            .                   E  
             2            .                   E  
             3            .                   E  
             4            .                   E

# EOF